## Tutorial 1: Corpora and Vector Spaces


In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [19]:
import os
import tempfile
TEMP_FOLDER = 'D:\OneDrive\ANLY580\datasets'
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "D:\OneDrive\ANLY580\datasets" will be used to save temporary dictionary and corpus.


### From Strings to Vectors

In [20]:
from gensim import corpora

In [21]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

First, let’s tokenize the documents, remove common words (using a toy stoplist) as well as words that only appear once in the corpus:

In [22]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


To convert documents to vectors, we’ll use a document representation called `bag-of-words`. In this representation, each document is represented by one vector where a vector element i represents the number of times the ith word appears in the document.  

It is advantageous to represent the questions only by their (integer) ids. The mapping between the questions and ids is called a dictionary:

In [23]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)

2017-10-21 21:35:44,903 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-10-21 21:35:44,905 : INFO : built Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...) from 9 documents (total 29 corpus positions)
2017-10-21 21:35:44,906 : INFO : saving Dictionary object under D:\OneDrive\ANLY580\datasets\deerwester.dict, separately None
2017-10-21 21:35:44,908 : INFO : saved D:\OneDrive\ANLY580\datasets\deerwester.dict


Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)


In [24]:
print(dictionary.token2id)

{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'user': 4, 'system': 5, 'response': 6, 'time': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [25]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(0, 1), (2, 1)]


The function `doc2bow()` simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a bag-of-words--a sparse vector, in the form of `[(word_id, word_count), ...]`.

In [26]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use
for c in corpus:
    print(c)

2017-10-21 21:35:48,327 : INFO : storing corpus in Matrix Market format to D:\OneDrive\ANLY580\datasets\deerwester.mm
2017-10-21 21:35:48,329 : INFO : saving sparse matrix to D:\OneDrive\ANLY580\datasets\deerwester.mm
2017-10-21 21:35:48,330 : INFO : PROGRESS: saving document #0
2017-10-21 21:35:48,331 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-10-21 21:35:48,333 : INFO : saving MmCorpus index to D:\OneDrive\ANLY580\datasets\deerwester.mm.index


[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (4, 1), (5, 1), (8, 1)]
[(0, 1), (5, 2), (8, 1)]
[(4, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(3, 1), (10, 1), (11, 1)]


### Corpus Streaming – One Document at a Time
Let’s assume the documents are stored in a file on disk, one document per line. Gensim only requires that a corpus be able to return one document vector at a time:

In [27]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('D:\OneDrive\ANLY580\datasets\mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [28]:
corpus_memory_friendly = MyCorpus() # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [29]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
[(1, 1), (4, 1), (8, 1)]
[(0, 1), (5, 1)]
[(4, 1), (6, 1), (7, 1)]
[]
[(10, 1)]
[(9, 1), (11, 1)]
[(11, 1)]


In [30]:
from six import iteritems

# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('D:\OneDrive\ANLY580\datasets\mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

# remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids + once_ids)
print(dictionary)

2017-10-21 21:35:54,721 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-10-21 21:35:54,723 : INFO : built Dictionary(51 unique tokens: ['"human', 'machine', 'interface', 'for', 'lab']...) from 9 documents (total 69 corpus positions)


Dictionary(9 unique tokens: ['interface', 'computer', 'user', 'system', 'response']...)


### Corpus Formats
One of the more notable file formats is the Matrix Market format. To save a corpus in the Matrix Market format:

In [31]:
# create a toy corpus of 2 documents, as a plain Python list
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.mm'), corpus)

2017-10-21 21:35:56,041 : INFO : storing corpus in Matrix Market format to D:\OneDrive\ANLY580\datasets\corpus.mm
2017-10-21 21:35:56,043 : INFO : saving sparse matrix to D:\OneDrive\ANLY580\datasets\corpus.mm
2017-10-21 21:35:56,044 : INFO : PROGRESS: saving document #0
2017-10-21 21:35:56,045 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2017-10-21 21:35:56,049 : INFO : saving MmCorpus index to D:\OneDrive\ANLY580\datasets\corpus.mm.index


In [32]:
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'corpus.mm'))

2017-10-21 21:36:19,279 : INFO : loaded corpus index from D:\OneDrive\ANLY580\datasets\corpus.mm.index
2017-10-21 21:36:19,280 : INFO : initializing corpus reader from D:\OneDrive\ANLY580\datasets\corpus.mm
2017-10-21 21:36:19,282 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


In [33]:
print(corpus)

MmCorpus(2 documents, 2 features, 1 non-zero entries)


In [34]:
# one way of printing a corpus: load it entirely into memory
print(list(corpus))  # calling list() will convert any sequence to a plain Python list

[[(1, 0.5)], []]


In [35]:
# another way of doing it: print one document at a time, making use of the streaming interface
for doc in corpus:
    print(doc)

[(1, 0.5)]
[]


### Compatibility with NumPy and SciPy

In [36]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5,2])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix_dense = gensim.matutils.corpus2dense(corpus, num_terms=10)

In [37]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)